In [3]:
pwd

'c:\\Users\\승범 pc\\Desktop\\공부\\딥러닝\\엘지에이머스'

In [18]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from phase2_module.src.utils.top_score_instance import check_the_score
from tqdm import tqdm
# 레이블 인코딩할 칼럼들
df_train_data = pd.read_csv("phase2_module/data/train.csv") # 학습용 데이터
df_test_data = pd.read_csv("phase2_module/data/test.csv") # 테스트 데이터(제출파일의 데이터)
drop_columns =['customer_country.1']
df_train_data = df_train_data.drop(columns=drop_columns)
df_test_data=df_test_data.drop(columns=drop_columns)
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train_data[label_columns], df_test_data[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
for col in label_columns:  
    df_train_data[col] = df_all.iloc[: len(df_train_data)][col]
    df_test_data[col] = df_all.iloc[len(df_train_data) :][col]
    
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[]}
train_result = {"f1":[], "precision":[], "recall":[]}
X = df_train_data.fillna(0).drop(["is_converted"],axis = 1)
Y = df_train_data['is_converted']

model = DecisionTreeClassifier(random_state=42)

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]
    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

bant_submit                float64
customer_country             int64
business_unit                int64
com_reg_ver_win_rate       float64
customer_idx                 int64
customer_type                int64
enterprise                   int64
historical_existing_cnt    float64
id_strategic_ver           float64
it_strategic_ver           float64
idit_strategic_ver         float64
customer_job                 int64
lead_desc_length             int64
inquiry_type                 int64
product_category             int64
product_subcategory          int64
product_modelname            int64
customer_position            int64
response_corporate           int64
expected_timeline            int64
ver_cus                      int64
ver_pro                      int64
ver_win_rate_x             float64
ver_win_ratio_per_bu       float64
business_area                int64
business_subarea             int64
lead_owner                   int64
dtype: object


5it [00:01,  2.62it/s]

----[K-Fold Train Score]-----
f1 score : 0.9982 / STD: (+/- 0.0001)
precision score : 0.9968 / STD: (+/- 0.0002)
recall score : 0.9996 / STD: (+/- 0.0003)
----[K-Fold Validation Score]-----
f1 score : 0.7790 / STD: (+/- 0.0100)
precision score : 0.7858 / STD: (+/- 0.0101)
recall score : 0.7725 / STD: (+/- 0.0127)


## 점수 확인
* customer_country.1만 제거
* 중복 제거 안함.

In [25]:
s = df_test_data.drop(['id','is_converted'],axis = 1)
pred = model.predict(s.fillna(0))
df_test_data['is_converted'] = pred
df_test_data.to_csv("submission.csv",index = False)
0.4222704266088214

In [27]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from phase2_module.src.utils.top_score_instance import check_the_score
from tqdm import tqdm
# 레이블 인코딩할 칼럼들
df_train_data = pd.read_csv("phase2_module/data/train.csv") # 학습용 데이터
df_test_data = pd.read_csv("phase2_module/data/test.csv") # 테스트 데이터(제출파일의 데이터)
drop_columns=['com_reg_ver_win_rate', 'id_strategic_ver', 'idit_strategic_ver', 'it_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x','customer_country.1', 'ver_win_ratio_per_bu']

df_train_data = df_train_data.drop(columns=drop_columns)
df_test_data=df_test_data.drop(columns=drop_columns)
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train_data[label_columns], df_test_data[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
for col in label_columns:  
    df_train_data[col] = df_all.iloc[: len(df_train_data)][col]
    df_test_data[col] = df_all.iloc[len(df_train_data) :][col]
    
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[]}
train_result = {"f1":[], "precision":[], "recall":[]}
X = df_train_data.fillna(0).drop(["is_converted"],axis = 1)
Y = df_train_data['is_converted']

model = DecisionTreeClassifier(random_state=42)

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]
    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:01,  3.12it/s]

----[K-Fold Train Score]-----
f1 score : 0.9982 / STD: (+/- 0.0001)
precision score : 0.9968 / STD: (+/- 0.0002)
recall score : 0.9996 / STD: (+/- 0.0003)
----[K-Fold Validation Score]-----
f1 score : 0.7837 / STD: (+/- 0.0071)
precision score : 0.7889 / STD: (+/- 0.0101)
recall score : 0.7787 / STD: (+/- 0.0082)


## 점수 확인 
* 파생 변수 컬럼 제거
* 중복 제거 안함

In [28]:
s = df_test_data.drop(['id','is_converted'],axis = 1)
pred = model.predict(s.fillna(0))
df_test_data['is_converted'] = pred
df_test_data.to_csv("submission.csv",index = False)
0.44951590594744123

In [29]:
def label_rows(row):
    if row['product_modelname'] is np.nan and row['product_subcategory'] is np.nan and row['product_category'] is np.nan:
        return 8
    elif row['product_modelname'] is np.nan and row['product_subcategory'] is np.nan:
        return 7
    elif row['product_modelname'] is np.nan and row['product_category'] is np.nan:
        return 6
    elif row['product_subcategory'] is np.nan and row['product_category'] is np.nan:
        return 5
    elif row['product_modelname'] is np.nan:
        return 4
    elif row['product_subcategory'] is np.nan:
        return 3
    elif row['product_category'] is np.nan:
        return 2
    else:
        return 1

In [46]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from phase2_module.src.utils.top_score_instance import check_the_score
from tqdm import tqdm
# 레이블 인코딩할 칼럼들
df_train_data = pd.read_csv("phase2_module/data/train.csv") # 학습용 데이터
df_test_data = pd.read_csv("phase2_module/data/test.csv") # 테스트 데이터(제출파일의 데이터)
drop_columns=['com_reg_ver_win_rate', 'id_strategic_ver', 'idit_strategic_ver', 'it_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x','customer_country.1', 'ver_win_ratio_per_bu']

df_train_data = df_train_data.drop(columns=drop_columns)
df_test_data=df_test_data.drop(columns=drop_columns)
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train_data[label_columns], df_test_data[label_columns]])
df_all['customer_interest'] = df_all.apply(lambda row: label_rows(row), axis=1)
label_columns.append('customer_interest')
for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
for col in label_columns:  
    df_train_data[col] = df_all.iloc[: len(df_train_data)][col]
    df_test_data[col] = df_all.iloc[len(df_train_data) :][col]
    
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[]}
train_result = {"f1":[], "precision":[], "recall":[]}
X = df_train_data.fillna(0).drop(["is_converted"],axis = 1)
Y = df_train_data['is_converted']

model = DecisionTreeClassifier(random_state=42)

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]
    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:01,  2.96it/s]

----[K-Fold Train Score]-----
f1 score : 0.9982 / STD: (+/- 0.0001)
precision score : 0.9968 / STD: (+/- 0.0002)
recall score : 0.9996 / STD: (+/- 0.0003)
----[K-Fold Validation Score]-----
f1 score : 0.7847 / STD: (+/- 0.0065)
precision score : 0.7905 / STD: (+/- 0.0077)
recall score : 0.7790 / STD: (+/- 0.0076)


## 점수 확인
* 내 Customer interest만 추가
* 이것도 제출했다 생각했는데 사실 추가하지 않은 상태로 제출되었음.

In [45]:
s = df_test_data.drop(['id','is_converted'],axis = 1)
pred = model.predict(s.fillna(0))
df_test_data['is_converted'] = pred
df_test_data.to_csv("submission.csv",index = False)
0.44951590594744123

0.44951590594744123